# AI Agents Crash Course - Part 2: Implementation Notebook

This notebook demonstrates the advanced concepts from Part 2:
- Modular crew architecture with YAML configuration
- Structured output generation with Pydantic
- Custom tool development

**Prerequisites:**
1. Install requirements: `pip install -r requirements.txt`
2. Set up `.env` file with your API keys
3. Ensure all project files are in the correct structure

## 1. Setup and Environment

In [3]:
# Install dependencies (uncomment if needed)
# !pip install crewai crewai-tools pydantic python-dotenv requests PyYAML

In [ ]:
import os
from dotenv import load_dotenv
from crewai import LLM

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## 2. YAML Configuration Overview

Let's examine the YAML configuration files that separate our agent definitions from code logic.

In [ ]:
import yaml

# Load and display agent configuration
with open('crews/config/agents.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)

print("📋 Available Agents:")
for agent_name, config in agents_config.items():
    print(f"\n🤖 {agent_name}:")
    print(f"   Role: {config['role']}")
    print(f"   Goal: {config['goal'][:100]}..." if len(config['goal']) > 100 else f"   Goal: {config['goal']}")

In [ ]:
# Load and display task configuration
with open('crews/config/tasks.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("📋 Available Tasks:")
for task_name, config in tasks_config.items():
    print(f"\n📝 {task_name}:")
    print(f"   Agent: {config['agent']}")
    description = config['description'][:150] + "..." if len(config['description']) > 150 else config['description']
    print(f"   Description: {description}")
    if 'depends_on' in config:
        print(f"   Depends on: {config['depends_on']}")

## 3. Basic Research Crew Implementation

Let's start with the basic research crew from the article.

In [ ]:
# Import our custom research crew
from crews.research_crew import ResearchCrew

# Create research crew instance
research_crew = ResearchCrew(llm = llm)
print("✅ Research crew initialized!")

In [ ]:
# Run a research task
topic = "The impact of AI on job markets"

print(f"🔍 Researching topic: {topic}")
print("This may take a few minutes...\n")

# Create and run the crew
result = research_crew.crew().kickoff(inputs={"topic": topic})

In [ ]:
from IPython.display import Markdown, display


# Display the research result
display(Markdown(result.raw))

## 4. Structured Output with Pydantic

Now let's demonstrate structured output generation using Pydantic schemas.

In [10]:
from crewai import Agent

from pydantic import BaseModel, Field

class EntityRelationEntity(BaseModel):
    entity: str = Field(description="The first entity in the triplet")
    relation: str = Field(description="The relation between the first and second entity")
    entity: str = Field(description="The second entity in the triplet")


In [11]:
from crewai import Agent

agent = Agent(
    role="Senior Linguist",
    goal="Analyse the query and extract entity-relation-entity triplets",
    backstory="You are a senior linguist that is known for your analytical skills.",
    verbose=True,
    llm=llm
)

In [12]:
from crewai import Task

task = Task(
    description="""Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: {query}
                   """,
    expected_output="""A structured JSON object with the
                       entity-relation-entity triplets""",
    output_pydantic=EntityRelationEntity,
    verbose=True,
    agent=agent
)

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

response = crew.kickoff(inputs={"query": "Paris is the capital of France."})


In [ ]:
print(response.raw)

## 5. Custom Tools Development

Let's explore the custom currency conversion tool.

In [15]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

In [16]:
class CurrencyConverterInput(BaseModel):
    """Input schema for CurrencyConverterTool."""
    amount: float = Field(..., description="The amount to convert.")
    from_currency: str = Field(..., description="The source currency code (e.g., 'USD').")
    to_currency: str = Field(..., description="The target currency code (e.g., 'EUR').")


In [17]:
class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another."
    args_schema: Type[BaseModel] = CurrencyConverterInput
    api_key: str = os.getenv("EXCHANGE_RATE_API_KEY") 

    def _run(self, amount: float, from_currency: str, to_currency: str) -> str:
        url = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{from_currency}"
        response = requests.get(url)
        
        if response.status_code != 200:
            return "Failed to fetch exchange rates."

        data = response.json()
        if "conversion_rates" not in data or to_currency not in data["conversion_rates"]:
            return f"Invalid currency code: {to_currency}"

        rate = data["conversion_rates"][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equivalent to {converted_amount:.2f} {to_currency}."


In [18]:
from crewai import Agent

currency_analyst = Agent(
    role="Currency Analyst",
    goal="Provide real-time currency conversions and financial insights.",
    backstory=(
        "You are a finance expert with deep knowledge of global exchange rates."
        "You help users with currency conversion and financial decision-making."
    ),
    tools=[CurrencyConverterTool()],  # Attach our custom tool
    verbose=True,
    llm=llm
)

In [19]:
from crewai import Task

currency_conversion_task = Task(
    description=(
        "Convert {amount} {from_currency} to {to_currency} "
        "using real-time exchange rates."
        "Provide the equivalent amount and "
        "explain any relevant financial context."
    ),
    expected_output=("A detailed response including the "
                     "converted amount and financial insights."),
    agent=currency_analyst
)

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[currency_analyst],
    tasks=[currency_conversion_task],
    process=Process.sequential
)

response = crew.kickoff(inputs={"amount": 100, 
                                "from_currency": "USD",
                                "to_currency": "EUR"})


In [ ]:
from IPython.display import Markdown, display


# Display the research result
display(Markdown(response.raw))